In [1]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook, reset_output, curdoc
from bokeh.embed import file_html
from bokeh.models import LabelSet, Label, ColumnDataSource, FactorRange
from bokeh.transform import factor_cmap
import bokeh.plotting as bp
from bokeh.layouts import row, column
from bokeh.models.widgets import Slider, TextInput
from bokeh.transform import cumsum
from bokeh.palettes import Category10

In [2]:
data = pd.read_csv("./MC1/mc1-reports-data.csv",parse_dates=True,infer_datetime_format=True, index_col=0)
data = data.sort_index()
data = data.fillna(0)
data.head()

,sewer_and_water,power,roads_and_bridges,medical,buildings,shake_intensity,location
time,,,,,,,
2020-04-06,2.0,4.0,1.0,0.0,4.0,0.0,17
2020-04-06,2.0,8.0,5.0,0.0,3.0,0.0,2
2020-04-06,9.0,7.0,5.0,0.0,4.0,0.0,8
2020-04-06,10.0,2.0,9.0,0.0,10.0,0.0,4
2020-04-06,6.0,2.0,9.0,0.0,4.0,1.0,15


#### Construcción dataset
- Se costruyen dos dataset, uno que contiene todos los datos para graficar los boxplot y otro para graficar los outliers.

In [12]:
l = 1
d = 8
data.loc[data["location"]==1].loc[data.loc[data["location"]==1].index.day==8]["shake_intensity"].max()

3.0

In [4]:
names = ["Palace Hills", "Northwest","Old Town", "Safe Town","Southwest","Downtown",
                       "Wilson Forest", "Scenic Vista", "Broadview", "Chapparal", "TerrapinSprings",
                       "Pepper Mill", "CheddardFord", "Easton", "Weston", "Southon", "Oak Willow",
                       "East Parton", "West Parton"]
localidade = []
tipo_daño = []
ldays = []
q1 = []
q2 = []
q3 = []
upper = []
lower = []
t_daño = ["sewer_and_water", "power", "roads_and_bridges", "medical", "buildings", "shake_intensity"]
for l in range(1,20):
    for d in range(6,12):
        for t in t_daño:
            localidade.append(names[l-1])
            tipo_daño.append(t)
            ldays.append(d)
            aux_q1 = data.loc[data["location"]==l].loc[data.loc[data["location"]==l].index.day==d][t].quantile(.25)
            aux_q2 = data.loc[data["location"]==l].loc[data.loc[data["location"]==l].index.day==d][t].quantile(.5)
            aux_q3 = data.loc[data["location"]==l].loc[data.loc[data["location"]==l].index.day==d][t].quantile(.75)
            q1.append(aux_q1)
            q2.append(aux_q2)
            q3.append(aux_q3)
            upper.append(aux_q3 + 1.5*(aux_q3-aux_q2))
            lower.append(aux_q1 - 1.5*(aux_q3-aux_q2))

In [5]:
len(localidade), len(tipo_daño), len(q1), len(q2), len(q3), len(upper), len(lower)

(684, 684, 684, 684, 684, 684, 684)

In [6]:
df_bxplt = pd.DataFrame()
df_bxplt["location"] = localidade
df_bxplt["day"] = ldays
df_bxplt["tipo"] = tipo_daño
df_bxplt["q1"] = q1
df_bxplt["q2"] = q2
df_bxplt["q3"] = q3
df_bxplt["upper"] = upper
df_bxplt["lower"] = lower
df_bxplt.head()

,location,day,tipo,q1,q2,q3,upper,lower
0,Palace Hills,6,sewer_and_water,0.0,4.0,8.0,14.0,-6.0
1,Palace Hills,6,power,0.0,2.0,7.0,14.5,-7.5
2,Palace Hills,6,roads_and_bridges,0.0,4.0,8.0,14.0,-6.0
3,Palace Hills,6,medical,1.0,4.0,8.0,14.0,-5.0
4,Palace Hills,6,buildings,0.0,4.0,8.0,14.0,-6.0


In [ ]:
#Para las localidades que quedaron sin datos
df_bxplt = df_bxplt.fillna(0)

In [21]:
import numpy as np
import pandas as pd

from bokeh.plotting import figure, show, output_file, output_notebook
output_notebook()
# generate some synthetic time series for six different categories
cats = list("abcdef")
yy = np.random.randn(2000)
g = np.random.choice(cats, 2000)
for i, l in enumerate(cats):
    yy[g == l] += i // 2
df = pd.DataFrame(dict(score=yy, group=g))

# find the quartiles and IQR for each category
groups = df.groupby('group')
q1 = groups.quantile(q=0.25)
q2 = groups.quantile(q=0.5)
q3 = groups.quantile(q=0.75)
iqr = q3 - q1
upper = q3 + 1.5*iqr
lower = q1 - 1.5*iqr

# find the outliers for each category
def outliers(group):
    cat = group.name
    return group[(group.score > upper.loc[cat]['score']) | (group.score < lower.loc[cat]['score'])]['score']
out = groups.apply(outliers).dropna()

# prepare outlier data for plotting, we need coordinates for every outlier.
if not out.empty:
    outx = []
    outy = []
    for keys in out.index:
        outx.append(keys[0])
        outy.append(out.loc[keys[0]].loc[keys[1]])

p = figure(tools="hover,box_select,pan,wheel_zoom,box_zoom,reset,save",
           background_fill_color="#efefef", x_range=cats, toolbar_location="right")

# if no outliers, shrink lengths of stems to be no longer than the minimums or maximums
qmin = groups.quantile(q=0.00)
qmax = groups.quantile(q=1.00)
upper.score = [min([x,y]) for (x,y) in zip(list(qmax.loc[:,'score']),upper.score)]
lower.score = [max([x,y]) for (x,y) in zip(list(qmin.loc[:,'score']),lower.score)]

# stems
p.segment(cats, upper.score, cats, q3.score, line_color="black")
p.segment(cats, lower.score, cats, q1.score, line_color="black")

# boxes
p.vbar(cats, 0.7, q2.score, q3.score, fill_color="#E08E79", line_color="black")
p.vbar(cats, 0.7, q1.score, q2.score, fill_color="#3B8686", line_color="black")

# whiskers (almost-0 height rects simpler than segments)
p.rect(cats, lower.score, 0.2, 0.01, line_color="black")
p.rect(cats, upper.score, 0.2, 0.01, line_color="black")

# outliers
if not out.empty:
    p.circle(outx, outy, size=6, color="#F38630", fill_alpha=0.6)

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = "white"
p.grid.grid_line_width = 2
p.xaxis.major_label_text_font_size="12pt"

#output_file("boxplot.html", title="boxplot.py example")

show(p)

Loading BokehJS ...

In [22]:
q3

0.75,score
group,
a,0.660623
b,0.570417
c,1.642768
d,1.728144
e,2.671889
f,2.768262
